In [25]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
import pandas as pd

import scipy.interpolate as interp
import astropy.constants as const

from multiprocessing import Pool
import emcee
import time

$\textbf{Note!}$ The folder containing this notebook should be in the same directory as the folder containing the ``custom-complete`` tool for calculating Kepler survey completeness in bins of stellar properties. You can download that tool [here](https://github.com/gbergsten/custom-completeness).

In [2]:
import sys

sys.path.append('../Completeness/')
from completeness import *

# Functions

In [3]:
# Shorthand for returning period and radius limits
def GetRange():
    Pmin, Pmax = 2, 100
    Rmin, Rmax = 1.0, 3.5

    return Pmin, Pmax, Rmin, Rmax

In [4]:
# Handy shorthand function for making log-spaced grids to integrate with
def log_grid(nx=100,ny=100):
    Pmin,Pmax,Rmin,Rmax = GetRange()
    
    x= np.geomspace(Pmin,Pmax,nx)
    y= np.geomspace(Rmin,Rmax,ny)
    P,R= np.meshgrid(x,y)

    logspace_x = np.exp((np.log(x[nx-1])-np.log(x[0]))/(nx-1))
    logspace_y = np.exp((np.log(y[ny-1])-np.log(y[0]))/(ny-1))
    dP,dR = P*(logspace_x-1), R*(logspace_y-1)
    
    return P,R,dP,dR,x,y

In [5]:
# A way to scale the radius valley based on host star mass (using scaling from Wu 2019).

def ScaledRadius(R=2, M=1, inverse=False):
    ### Radius given in Earth radii
    ### Mass given in solar masses
    
    power = 1/4
    if inverse:
        power = -power

    Rprime = R * (M)**(power)
    return Rprime

Rsplit = ScaledRadius()

## PLDF: dF / dRdP

In [6]:
# Shape Function (describes behavior of planet population)

### Free parameter arrangement:
### F0: Avg. # of planets per star
### P_break: Break in period for power law
### beta_1: Exponent prior to P_break
### beta_2: Exponent after P_break
### P_central: Center of hyperbolic tangent between sE dominance and sN dominance
### chi_1: Ratio of sE to sN prior to P_central
### chi_2: Ratio of sE to sN after P_central
### s: smoothness of tanh

def g(P,R,fparam):
        
    F0, P_break, beta_1, beta_2, P_central, chi_1, chi_2, s = fparam
    
    # Use a broken power law to calculate overall occurrence
    occ_SEMN = (P/P_break)**np.where(P<P_break, beta_1, beta_2)
    
    # Use a hyperbolic tangent which changes with period
    sx = 0.5 + 0.5*np.tanh((np.log10(P)-np.log10(P_central))/np.log10(s))
    # Normalize sides of the tanh curve to chi_1 and chi_2
    X = chi_1*(1-sx) + sx*(chi_2)
    
    # Some normalization to incorporate this form into the shape function
    a,b,c = np.log(Rmin), np.log(Rsplit), np.log(Rmax)
    g2 = X*(c-b) / ((b-a) + X*(c-b) - X*(b-a))
    
    # Provide g2 for super-Earths, the complement 1-g2 for sub-Neptunes
    g_split = np.where(R<Rsplit, g2, 1-g2)
        
    # Divide the broken power law's occurrence amongst the super-Earths and/or sub-Neptunes.
    g = g_split/R * occ_SEMN 
        
    return g

In [7]:
intP,intR,dP,dR,x,y = log_grid()

# Normalization parameter for the PLDF (as in Burke et al. 2015).
# Ensures that the product of Cn * g equals to 1 when integrated over the P,R range used.
def function_Cn(fparam):
    
    Cn = 1 / np.sum(g(intP,intR,fparam) * dP * dR)
    
    return Cn

In [8]:
# The full PLDF to give differential occurrence rates, defined as df / (dP dR) 
# When integrated over the P,R range used, equals the number of planets per star F0.

def df(P,R,fparam):
        
    F0 = fparam[0]
    Cn = function_Cn(fparam)
    
    df = F0 * Cn * g(P,R,fparam)
    
    return df

## Evaluating the Model Likelihood
Following the equations of Burke et al. (2015) and Youdin (2011).

Calculate N_exp (the predicted number of detections), which involves scaling the shape function $g$ by the survey completeness.



In [9]:
def function_Nexp(fparam):
    
    F0 = fparam[0]
    Cn = function_Cn(fparam)

    integrand = OneRunCompletenessGrid * g(intP,intR,fparam) * dP * dR
        
    integral = np.sum(OneRunCompletenessGrid * g(intP,intR,fparam) * dP * dR)
    
    Nexp = F0 * Cn * integral
    return Nexp

Shorthand for returning the number of planets being fit (i.e., length of the P and R data arrays).

In [10]:
def function_Npl(P,R):
    assert len(P) == len(R)
    Npl = len(P)
    return Npl

Calculate the log likelihood of the given parameters to match the data through the PLDF.

In [11]:
### NOTE: Here fparam is passed FIRST, rather than last.
### This is a workaround for the order of args scipy.minimze takes.

def lnL(fparam,P,R):
    Npl = function_Npl(P,R)
    Nexp = function_Nexp(fparam)
    
    F0 = fparam[0]
    Cn = function_Cn(fparam)
    
    g_term = np.sum(np.log(g(P,R,fparam)))
    
    lnL = -Nexp + Npl*np.log(F0) + Npl*np.log(Cn) + g_term
    
    return lnL

### Priors

This part mostly just follows the emcee documentation.
As such, it's likely where things can go wrong.
https://emcee.readthedocs.io/en/stable/tutorials/line/#

In [12]:
def log_prior(fparam):
    
    F0, P_break, beta_1, beta_2, P_cent, chi_1, chi_2, s = fparam
        
    # Priors are generally uniform with ranges motivated by the data.
    # See Appendix C.2 of Bergsten et al. (2022) for more details.
    
    if \
    0.05 < F0 < 5 and\
    2< P_break < 20 and\
    -2 < beta_1 < 5 and\
    -2 < beta_2 < 5 and\
    4 < P_cent < 50 and\
    (1-chi_1) < chi_1 and\
    0 < chi_1 < 1 and\
    0 < chi_2 < 0.5 and\
    1 < s < 5 and\
    np.log10(2*Pmin) <= np.log10(P_cent) - np.log10(s) and\
    np.log10(0.5*Pmax) >= np.log10(P_cent) + np.log10(s):
        return 0.0
    
    return -np.inf

In [13]:
### Combine with the log likelihood to create the log posterior

### NOTE: Here fparam is passed FIRST, rather than last.
### This is a workaround for the order of args emcee takes.

def log_posterior(fparam,P,R):
    
    lp = log_prior(fparam)
    
    if not np.isfinite(lp):
        return -np.inf
    
    return lp + lnL(fparam,P,R)

# Reading in Data/Files

### Reading in the Kepler Sample

In [14]:
# Return a subset of dwarf stars in the desired mass range.
def get_mass_range(stars, row):
    # Apply a cut to select only dwarf stars (in the style of Huber et al.)
    isdwarf= stars['logg'] > 1./4.671 * np.arctan((stars['Teff']-6300.)/-67.172)+ 3.876
    inBin = (stars['Mass'] >= row['Lo']) & (stars['Mass'] < row['Hi']) & isdwarf
    return stars[inBin]

In [15]:
### Reading in the Berger+2020a Gaia-Keppler stellar properties catalogue
gaia= Table.read('../Completeness/files/Berger2020a_GaiaKeplerCatalog_Table2.fits')
### Load in the set of stars with detection metrics (courtesy of Gijs)
metrics_list = np.load('../Completeness/files/metricslist.npz')
### Find the subset of stars with both detection metrics and Gaia data
gaia_kepler_subset = np.intersect1d(gaia['KIC'], metrics_list['KID'])
goodstars = np.isin(gaia['KIC'], gaia_kepler_subset)
kg = gaia[goodstars]

### Cut to select only dwarf stars (following Huber et al. 2016)
isdwarf = (kg['logg'] > 1./4.671 * np.arctan((kg['Teff']-6300.)/-67.172)+ 3.876)
kg = kg[isdwarf]

### Reading Info about Mass Bins

In [16]:
# Read in the file with info about ranges for equally populated bins of stellar mass
mbfilename = 'MassBins-EqualCounts_6bins.txt'
t = Table.read('files/{}'.format(mbfilename),\
                        format='csv', names=['Lo', 'Hi'], data_start=0)

In [19]:
# Read in planet properties catalog, including per-candidate reliability
KOIcatalog = pd.read_csv('files/dr25_KOIs_dr25.csv')
reliability = KOIcatalog.totalReliability

# Fitting the PLDF

In [20]:
# Some initial guesses to help with fitting
guess = [[0.680, 9.223, 0.639, -0.868, 11.419, 0.847, 0.395, 2.496],
         [0.952, 13.224, 0.220, -1.121, 7.555, 0.756, 0.342, 1.505],
         [0.742, 5.764, 1.199, -0.663, 11.013, 0.739, 0.375, 1.977],
         [0.667, 7.006, 0.907, -0.856, 13.205, 0.826, 0.278, 2.524],
         [0.656, 12.123, 0.404, -1.072, 16.804, 0.838, 0.327, 2.262],
         [0.537, 7.144, 1.833, -0.730, 17.025, 0.879, 0.388, 2.147]]

In [38]:
intP,intR,dP,dR,x,y = log_grid()
Pmin,Pmax,Rmin,Rmax = GetRange()

num_draws = 100
nstep = 10000

for this_row in range(len(t)):
    
    # Find which mass bin we're working in
    row = t[this_row]
    # Grab initial guesses for that bin
    initial=guess[this_row]
    
    ### Initialization of all walkers
    nwalk = 16
    np.random.seed(42)
    npos = nwalk
    pos = initial + 1e-4 * np.random.randn(nwalk, len(initial))
    
    #####################################
    
    thisString = str(row['Lo']) + '-' + str(row['Hi'])
    
    tracker = []
    unused = []
    
    # Middle of the mass bin
    Mavg = np.mean([row['Lo'], row['Hi']])
    
    # Calculate the average survey completeness for this bin of stellar mass
    Pgrid, Rgrid, completeness, nstars = get_completeness(ranges={'Mass':[row['Lo'],row['Hi']]})
    # Make an interpolation for the "total" survey completeness, approximated by (avg * nstars)
    SummedEtaInterp = interp.interp2d(Pgrid,Rgrid,completeness.T*nstars) 
    OneRunCompletenessGrid = SummedEtaInterp(x,y)
    
    #####################################
    
    ### Adjusting star and planet samples
    ### Mass cut.
    kg2 = get_mass_range(kg, row)
    
    ### Load up the Kepler dr25 survey
    planets_path = 'files/q1_q17_dr25_koi.tbl'
    koi = Table.read(planets_path, format='ipac')
    iscandidate= koi['koi_pdisposition']=='CANDIDATE'
    isreliable= koi['koi_score']>= 0.0 

    isGood = np.isin(koi['kepid'], kg2['KIC']) & iscandidate #& isreliable

    ### Get rid of the 3(?) planets where there's no transit depth info
    ### (required for the step of updating planet radii)
    isUseless = koi[isGood]['koi_ror'].mask
    print('There are {} planets we cant use bc they lack transit depth.'.format(np.sum(isUseless)))
    koi = koi[isGood][~isUseless]

    Kepler_P = koi['koi_period']
    Kepler_R = koi['koi_prad']

    ### Updating the planet radii based on Berger stellar radii
    updated_radii = []
    ### Also store the reliability to draw from later
    sample_reliability = []
    for p in koi:
        updated_radii.append((p['koi_ror']*kg[np.where(kg['KIC']==p['kepid'])]['Rad'].data*const.R_sun/const.R_earth).value[0])
        sample_reliability.append(KOIcatalog.iloc[np.where(KOIcatalog.kepoi_name==p['kepoi_name'])].totalReliability.values[0])
    Kepler_R = updated_radii
    
    ### Limiting the Kepler Sample to the desired P,R range.
    Kepler_P, Kepler_R = np.asarray(Kepler_P), np.asarray(Kepler_R)
    inRange = np.where((Pmin < Kepler_P) & (Pmax > Kepler_P) & (Rmin < Kepler_R) & (Rmax > Kepler_R))
    Kepler_P = Kepler_P[inRange]
    Kepler_R = Kepler_R[inRange]
    sample_reliability = np.array(sample_reliability)[inRange]
    
    ### Fully exclude things with zero reliability (typically the 100% false positives)
    isnonzeroReliability = (sample_reliability != 0)
    Kepler_P = Kepler_P[isnonzeroReliability]
    Kepler_R = Kepler_R[isnonzeroReliability]
    sample_reliability = sample_reliability[isnonzeroReliability]
    
    print('{} M: {} stars; {} planets'.format(thisString,len(kg2),len(Kepler_P)))
        
    #####################################
    
    ### Adjust the boundary between sE and mN by bin mass
    Mavg = np.mean([row['Lo'], row['Hi']])
    Rsplit = ScaledRadius(M=Mavg)
    print('The boundary between sE and mN is at {:.2f} Re.'.format(Rsplit))
    
    #####################################
    
    # Performing sample draws based on candidate reliability
    use_array = np.array([np.random.choice([True,False], size=num_draws, p=[rel, 1-rel]) for rel in sample_reliability])

    # Identifying any unused planets (things that have low (~<1%) reliability)
    # Sometimes a very-low-reliability candidate is never incorporated, 
    # we opted to let the reliability speak for itself (i.e., we don't do anything with the ignored ones)
    if np.any([np.all(use_array[i,:]==False) for i in range(len(sample_reliability))]):
        print('{} planets were never used!'.format(len(np.where([np.all(use_array[i,:]==False) for i in range(len(sample_reliability))])[0])))
        unused = np.where([np.all(use_array[i,:]==False) for i in range(len(sample_reliability))])[0]
    print('Unused Indices:', unused)
    
    ### MCMC Steps
    nwalkers, ndim = pos.shape
    timestr = time.strftime("%Y.%m.%d-%H:%M:%S")
     
    # Storing info about the unused candidates just for bookkeeping
    if len(unused) != 0:
        print('Reliability of unused candidates:', sample_reliability[unused])
        np.savetxt('files/MCMC_Outputs/unused_flag_{}M_v'.format(thisString.replace('.','p')) + timestr + ".csv",
                   unused, delimiter=',')
    
    # Filename for storing MCMC results - indexed by start time.
    filename = "files/MCMC_Outputs/Fit_{}M_v".format(thisString.replace('.','p')) + timestr 
    print(filename)
    
    for draw in range(num_draws):
        this_Kepler_P = Kepler_P[use_array[:,draw]]
        this_Kepler_R = Kepler_R[use_array[:,draw]]
        tracker.append([draw,len(this_Kepler_P)])
        
        this_draw_filename = filename + '_draw_' + str(draw).zfill(3) + ".h5"
        
        backend = emcee.backends.HDFBackend(this_draw_filename)
        backend.reset(nwalkers, ndim)
        with Pool() as pool:
            sampler = emcee.EnsembleSampler(nwalkers, ndim, log_posterior,
                                            args=(this_Kepler_P, this_Kepler_R),
                                            backend=backend, pool=pool)
            sampler.run_mcmc(pos, nstep, progress=True)
        
    np.savetxt('files/MCMC_Outputs/tracker_{}M_v'.format(thisString.replace('.','p')) + timestr + ".csv",
                   tracker, delimiter=',')
    
    print('Runs Completed.')
    print()
    print('---')
    print()